In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain_groq import ChatGroq

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
llm = ChatGroq(model="openai/gpt-oss-120b",
api_key=os.getenv("GROQ_API_KEY"),
temperature=0.5)

In [6]:
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001E49A15C2F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E49A15CD70>, model_name='openai/gpt-oss-120b', temperature=0.5, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
import PyPDF2


In [20]:
import langchain
from langchain_core.globals import set_debug

set_debug(True)

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
RESPONSE_JSON_STRING = json.dumps(RESPONSE_JSON, indent=2)

In [11]:
quiz_generation_prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """You are an expert MCQ maker. 
        It is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
        
        Make sure the questions are not repeated and check all the questions to be conforming the text as well.
        Make sure to format your response like the RESPONSE_JSON below and use it as a guide. 
        Ensure to make {number} MCQs.
        
        ### RESPONSE_JSON
        {response_json}"""
    ),
    (
        "human", 
        "Text: {text}"
    )
])

In [26]:
quiz_chain = quiz_generation_prompt | llm | StrOutputParser()

In [21]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# --- PROMPT 2: EVALUATION (Your new template) ---
review_prompt = ChatPromptTemplate.from_template(
    """You are an expert english grammarian and writer. 
    Given a Multiple Choice Quiz for {subject} students.
    You need to evaluate the complexity of the question and give a complete analysis of the quiz.
    
    Quiz_MCQs:
    {quiz}
    
    Check from an expert English Writer of the above quiz:"""
)

In [23]:
# Input: {subject, quiz} -> Output: String (The Review)
review_chain = review_prompt | llm | StrOutputParser()

In [27]:
generate_evaluate_chain = (
    # Step 1: Pass inputs through, but ALSO run quiz_chain and store result in 'quiz'
    RunnablePassthrough.assign(quiz=quiz_chain)
    
    # Step 2: Now that 'quiz' is in the state, run review_chain and store in 'review'
    | RunnablePassthrough.assign(review=review_chain)
)

In [28]:
file_path = r"C:\Users\Loba\automcqgen\data.txt"
file_path

'C:\\Users\\Loba\\automcqgen\\data.txt'

In [29]:
with open(file_path, "r") as file:
    TEXT = file.read()

In [30]:

NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [33]:
result = generate_evaluate_chain.invoke({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": RESPONSE_JSON_STRING # -> json.dumps(RESPONSE_JSON)
})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "text": "Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of thei

In [34]:
result

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [35]:
# Access the outputs
print("--- GENERATED QUIZ ---")
print(result['quiz'])

--- GENERATED QUIZ ---
{
  "1": {
    "mcq": "What does biology study?",
    "options": {
      "a": "The study of rocks",
      "b": "The study of life",
      "c": "The study of stars",
      "d": "The study of weather"
    },
    "correct": "The study of life"
  },
  "2": {
    "mcq": "Which major theme explains the unity and diversity of life?",
    "options": {
      "a": "Photosynthesis",
      "b": "Evolution",
      "c": "Respiration",
      "d": "Osmoregulation"
    },
    "correct": "Evolution"
  },
  "3": {
    "mcq": "What basic structural unit is found in all organisms?",
    "options": {
      "a": "Organ",
      "b": "Tissue",
      "c": "Cell",
      "d": "Molecule"
    },
    "correct": "Cell"
  },
  "4": {
    "mcq": "Which of the following is NOT listed as a subdiscipline of biology in the text?",
    "options": {
      "a": "Molecular biology",
      "b": "Anatomy",
      "c": "Astrophysics",
      "d": "Ecology"
    },
    "correct": "Astrophysics"
  },
  "5": {
  

In [36]:
print("\n--- EXPERT REVIEW ---")
print(result['review'])


--- EXPERT REVIEW ---
## QUICK SUMMARY  

| Question # | Bloom’s level* | Linguistic difficulty | Overall rating (1‑5) |
|------------|----------------|-----------------------|----------------------|
| 1          | Remembering    | Very low (simple noun phrase) | 1 |
| 2          | Remembering / Understanding | Low (single‑word options) | 1 |
| 3          | Remembering    | Low (basic scientific term) | 1 |
| 4          | Remembering / Application | Low‑moderate (requires knowledge of “sub‑discipline” list) | 2 |
| 5          | Remembering / Understanding | Low‑moderate (numeric phrase) | 2 |

\* **Bloom’s taxonomy** levels are used as a quick proxy for cognitive demand. All items sit at the *Recall* end of the spectrum; none require higher‑order reasoning, synthesis or evaluation.

---

## 1.  GENERAL OBSERVATIONS  

1. **Target Audience & Purpose**  
   - The quiz is clearly aimed at **introductory‑level biology students** (e.g., first‑year undergraduates or advanced high‑school lea

In [37]:
quiz = result.get("quiz")

In [38]:
quiz = json.loads(quiz)

In [39]:
quiz

{'1': {'mcq': 'What does biology study?',
  'options': {'a': 'The study of rocks',
   'b': 'The study of life',
   'c': 'The study of stars',
   'd': 'The study of weather'},
  'correct': 'The study of life'},
 '2': {'mcq': 'Which major theme explains the unity and diversity of life?',
  'options': {'a': 'Photosynthesis',
   'b': 'Evolution',
   'c': 'Respiration',
   'd': 'Osmoregulation'},
  'correct': 'Evolution'},
 '3': {'mcq': 'What basic structural unit is found in all organisms?',
  'options': {'a': 'Organ', 'b': 'Tissue', 'c': 'Cell', 'd': 'Molecule'},
  'correct': 'Cell'},
 '4': {'mcq': 'Which of the following is NOT listed as a subdiscipline of biology in the text?',
  'options': {'a': 'Molecular biology',
   'b': 'Anatomy',
   'c': 'Astrophysics',
   'd': 'Ecology'},
  'correct': 'Astrophysics'},
 '5': {'mcq': 'According to the passage, life on Earth emerged more than how many billion years ago?',
  'options': {'a': '1.5 billion years',
   'b': '2.5 billion years',
   'c': '

In [40]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [41]:
quiz_table_data

[{'MCQ': 'What does biology study?',
  'Choices': 'a: The study of rocks | b: The study of life | c: The study of stars | d: The study of weather',
  'Correct': 'The study of life'},
 {'MCQ': 'Which major theme explains the unity and diversity of life?',
  'Choices': 'a: Photosynthesis | b: Evolution | c: Respiration | d: Osmoregulation',
  'Correct': 'Evolution'},
 {'MCQ': 'What basic structural unit is found in all organisms?',
  'Choices': 'a: Organ | b: Tissue | c: Cell | d: Molecule',
  'Correct': 'Cell'},
 {'MCQ': 'Which of the following is NOT listed as a subdiscipline of biology in the text?',
  'Choices': 'a: Molecular biology | b: Anatomy | c: Astrophysics | d: Ecology',
  'Correct': 'Astrophysics'},
 {'MCQ': 'According to the passage, life on Earth emerged more than how many billion years ago?',
  'Choices': 'a: 1.5 billion years | b: 2.5 billion years | c: 3.7 billion years | d: 5.0 billion years',
  'Correct': '3.7 billion years'}]

In [42]:
quiz=pd.DataFrame(quiz_table_data)

In [43]:
quiz.to_csv("biologyquestions.csv", index=False)